In [481]:
import pandas as pd
from pymongo import MongoClient

In [482]:
MONGO_URI = "mongodb://localhost:27017"  
DATABASE_NAME = "data_query_db1"
COLLECTION_NAME = "csv_data_collection2"

csv_file_path = "sample_data.csv"  


In [483]:
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

data = pd.read_csv(csv_file_path)
data['Discount'] = data['Discount'].str.replace('%', '').astype(int)


data_dict = data.to_dict(orient="records")

result = collection.delete_many({}) 
result = collection.insert_many(data_dict)
print(f"{len(result.inserted_ids)} records inserted into MongoDB.")

10 records inserted into MongoDB.


In [484]:
for record in collection.find().limit(5):
    print(record)

{'_id': ObjectId('671bf92f9ad718c21d0ac98d'), 'ProductID': 101, 'ProductName': 'Wireless Mouse', 'Category': 'Electronics', 'Price': 25.99, 'Rating': 4.5, 'ReviewCount': 200, 'Stock': 150, 'Discount': 10, 'Brand': 'Logitech', 'LaunchDate': '15-01-2022'}
{'_id': ObjectId('671bf92f9ad718c21d0ac98e'), 'ProductID': 102, 'ProductName': 'Gaming Keyboard', 'Category': 'Electronics', 'Price': 75.49, 'Rating': 4.7, 'ReviewCount': 350, 'Stock': 85, 'Discount': 5, 'Brand': 'Corsair', 'LaunchDate': '20-11-2021'}
{'_id': ObjectId('671bf92f9ad718c21d0ac98f'), 'ProductID': 103, 'ProductName': 'Noise Cancelling Headphones', 'Category': 'Electronics', 'Price': 199.99, 'Rating': 4.8, 'ReviewCount': 125, 'Stock': 60, 'Discount': 15, 'Brand': 'Sony', 'LaunchDate': '05-09-2021'}
{'_id': ObjectId('671bf92f9ad718c21d0ac990'), 'ProductID': 104, 'ProductName': 'Running Shoes', 'Category': 'Sports', 'Price': 49.99, 'Rating': 4.3, 'ReviewCount': 500, 'Stock': 200, 'Discount': 20, 'Brand': 'Nike', 'LaunchDate': '

In [485]:
with open('api_key.txt', 'r') as file:
    groq_api_key = file.read().strip()  


from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.1-70b-versatile"
)

print("LLM initialized successfully.")


LLM initialized successfully.


In [534]:
#user_input = "Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand 'Nike' or 'Sony'."
user_input ="Which products in the Electronics category have a rating of 4.5 or higher and are in stock?"
#user_input = "List products launched after January 1, 2022, in the Home & Kitchen or Sports categories with a discount of 10% or more, sorted by price in descending order."
print(f"User Query: {user_input}")

User Query: Which products in the Electronics category have a rating of 4.5 or higher and are in stock?


In [535]:
from langchain_core.prompts import PromptTemplate

query_template = PromptTemplate(
    input_variables=["user_input"],
    template=
    '''
    Generate a MongoDB query based on the user's request: "{user_input}". 
    ### Instructions: 
    - Return only the MongoDB query as a string that can be directly fed into MongoDB. 
    - The output should be syntactically correct for MongoDB operations such as find(), findOne(), aggregate(), etc.
    - Use the following schema for reference:
    | ProductID | ProductName | Category | Price | Rating | ReviewCount | Stock | Discount | Brand | LaunchDate(DD-MM-YYYY) |
    - note that we will be using the query not in javascript but in python so return accordingly and only provide the entire query starting from db.collections.
    in string format 
    
    
    - The output should reflect valid MongoDB query syntax using double ("") and single ('') quotes where necessary.
    - do not provide multiple queries only provide one best and optimum query
    - Return only the MongoDB query as a string that can be directly fed into MongoDB
    - trim the string provide only the query
    - Ensure that all dates are formatted as Python datetime objects using the format: 
      datetime.datetime(year, month, day, hour=0, minute=0, second=0).
    ensure all the above points are followed
    '''
)
llm_chain = LLMChain(llm=llm, prompt=query_template)

In [536]:
generated_query = llm_chain.invoke(user_input)
print(generated_query)


{'user_input': 'Which products in the Electronics category have a rating of 4.5 or higher and are in stock?', 'text': '```\ndb.collection.find({"Category": "Electronics", "Rating": {"$gte": 4.5}, "Stock": {"$gt": 0}})\n```'}


In [537]:
mongo_query = generated_query['text'].strip('`') 

print(mongo_query)
type(mongo_query)



db.collection.find({"Category": "Electronics", "Rating": {"$gte": 4.5}, "Stock": {"$gt": 0}})



str

In [538]:
print(mongo_query)


db.collection.find({"Category": "Electronics", "Rating": {"$gte": 4.5}, "Stock": {"$gt": 0}})



In [539]:
print(mongo_query)


db.collection.find({"Category": "Electronics", "Rating": {"$gte": 4.5}, "Stock": {"$gt": 0}})



In [547]:
import re
from datetime import datetime

iso_date_pattern = r'ISODate\("([\d\-T:\.Z]+)"\)'

def iso_to_datetime(iso_string):
    try:
        return datetime.strptime(iso_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    except ValueError:
        return datetime.strptime(iso_string, "%Y-%m-%dT%H:%M:%SZ")

if re.search(iso_date_pattern, mongo_query):
    matches = re.findall(iso_date_pattern, mongo_query)
    
    converted_dates = [iso_to_datetime(match) for match in matches]
    
    
    for match in matches:
        mongo_query = mongo_query.replace(f'ISODate("{match}")', f'"{iso_to_datetime(match)}"')  

    print("Converted dates:", converted_dates)
    print("Updated mongo_query:", mongo_query)
else:
    print("No ISO dates found in the mongo_query.")


No ISO dates found in the mongo_query.


In [548]:
import re

match = re.search(r'db\.collection\.(\w+)\((.*)\)', mongo_query)
if match:
    method = match.group(1)  
    arguments = match.group(2)  
else:
    raise ValueError("Invalid query format")

In [549]:
method

'find'

In [550]:
arguments

'{"Category": "Electronics", "Rating": {"$gte": 4.5}, "Stock": {"$gt": 0}}'

In [551]:
#import pandas as pd

try:
    if method == "aggregate":
        results = eval(f"collection.aggregate({arguments})")
    else:
        results = eval(f"collection.{method}({arguments})")

    documents = []

    if method == "find":
        results_list = list(results)  
        if results_list:
            print(f"Retrieved {len(results_list)} Document(s):")
            for idx, document in enumerate(results_list, start=1):
                print(f"Document {idx}:")
                documents.append(document)  
                for key, value in document.items():
                    print(f"  {key}: {value}")
                print()  
        else:
            print("No documents found.")
    elif method == "aggregate":
        results_list = list(results) 
        if results_list:
            print(f"Aggregation Results: {len(results_list)} Result(s):")
            for idx, doc in enumerate(results_list, start=1):
                print(f"Result {idx}:")
                documents.append(doc)  
                for key, value in doc.items():
                    print(f"  {key}: {value}")
                print()  
        else:
            print("No results from aggregation.")

    
    df = pd.DataFrame(documents)

    
    csv_file_name = "output_results.csv"
    df.to_csv(csv_file_name, index=False)

    print(f"Results saved to {csv_file_name}")

except Exception as e:
    print(f"An error occurred: {e}")

Retrieved 5 Document(s):
Document 1:
  _id: 671bf92f9ad718c21d0ac98d
  ProductID: 101
  ProductName: Wireless Mouse
  Category: Electronics
  Price: 25.99
  Rating: 4.5
  ReviewCount: 200
  Stock: 150
  Discount: 10
  Brand: Logitech
  LaunchDate: 15-01-2022

Document 2:
  _id: 671bf92f9ad718c21d0ac98e
  ProductID: 102
  ProductName: Gaming Keyboard
  Category: Electronics
  Price: 75.49
  Rating: 4.7
  ReviewCount: 350
  Stock: 85
  Discount: 5
  Brand: Corsair
  LaunchDate: 20-11-2021

Document 3:
  _id: 671bf92f9ad718c21d0ac98f
  ProductID: 103
  ProductName: Noise Cancelling Headphones
  Category: Electronics
  Price: 199.99
  Rating: 4.8
  ReviewCount: 125
  Stock: 60
  Discount: 15
  Brand: Sony
  LaunchDate: 05-09-2021

Document 4:
  _id: 671bf92f9ad718c21d0ac991
  ProductID: 105
  ProductName: Smartwatch
  Category: Electronics
  Price: 159.99
  Rating: 4.6
  ReviewCount: 220
  Stock: 45
  Discount: 10
  Brand: Apple
  LaunchDate: 30-03-2022

Document 5:
  _id: 671bf92f9ad718c2

In [552]:
query_template2 = PromptTemplate(
    input_variables=["df","user_input"],
    template=
    '''
   You are a data analysis assistant with expertise in generating insights from product data. 

### User Question:
{user_input}

### Data Overview:
Here are the details of the products retrieved based on the user's query:

{df}

### Task:
Based on the user’s question and the provided product data, generate a detailed insight summary. The insights should include:
1. The total number of products found that meet the criteria.
2. Key performance insights, including ratings, review counts, and pricing.
3. Brand performance details, focusing on the brands associated with the products.
4. A ranking or evaluation of the products based on the criteria given.
5. Suggestions or next steps for the user based on the findings.

Please ensure that the insights are clear, concise, and easy for the user to understand, avoiding technical jargon. Format your response in a structured manner similar to the example below:

### Example Insight Format:
- **Total Products Found:** X
- **Product Details:**
  - **Product Name:** [Product1 Name]
  - **Category:** [Product1 Category]
  - **Price:** $[Product1 Price]
  - **Rating:** [Product1 Rating]
  - **Review Count:** [Product1 Review Count]
  - **Brand:** [Product1 Brand]
  - **Launch Date:** [Product1 Launch Date]
  - *(Repeat for each product found)*
  
- **Key Insights:**
  1. **Overall Performance:** Based on the products found, we have a total of X products that meet your criteria of having a rating below 4.5, more than 200 reviews, and being offered by brands such as Nike or Sony.
  2. **Best Performing Product:** The product with the highest rating among those found is [Best Product Name], with a rating of [Best Product Rating] and [Best Product Review Count] reviews.
  3. **Lowest Performing Product:** Conversely, the product with the lowest rating is [Lowest Product Name], which has a rating of [Lowest Product Rating] and [Lowest Product Review Count] reviews.
  4. **Brand Analysis:** The brand [Most Common Brand] has the most products in this selection, indicating a strong presence in this category.
  
- **Next Steps:**
  - Consider exploring products with higher ratings for better quality options.
  - If looking for discounts, check if any of these products are currently on sale or promotion.
  - For further details or specific recommendations, feel free to ask!

    '''
)
llm_chain2 = LLMChain(llm=llm, prompt=query_template2)

In [546]:
if not df.empty:
    input_data = {
        "df": df.to_string(),
        "user_input": user_input
    }
    generated_insight = llm_chain2.invoke(input_data)  # Convert DataFrame to string
    print(generated_insight['text'])

### Insight Summary:

- **Total Products Found:** 5
- **Product Details:**
  - **Product Name:** Wireless Mouse
  - **Category:** Electronics
  - **Price:** $25.99
  - **Rating:** 4.5
  - **Review Count:** 200
  - **Brand:** Logitech
  - **Launch Date:** 15-01-2022

  - **Product Name:** Gaming Keyboard
  - **Category:** Electronics
  - **Price:** $75.49
  - **Rating:** 4.7
  - **Review Count:** 350
  - **Brand:** Corsair
  - **Launch Date:** 20-11-2021

  - **Product Name:** Noise Cancelling Headphones
  - **Category:** Electronics
  - **Price:** $199.99
  - **Rating:** 4.8
  - **Review Count:** 125
  - **Brand:** Sony
  - **Launch Date:** 05-09-2021

  - **Product Name:** Smartwatch
  - **Category:** Electronics
  - **Price:** $159.99
  - **Rating:** 4.6
  - **Review Count:** 220
  - **Brand:** Apple
  - **Launch Date:** 30-03-2022

  - **Product Name:** Smart TV
  - **Category:** Electronics
  - **Price:** $399.99
  - **Rating:** 4.7
  - **Review Count:** 150
  - **Brand:** Samsung
